<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%2013%20-%20Preparar%20un%20modelo%20de%20TensorFlow%20para%20móviles%20con%20TensorFlow%20Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1: Instalar las dependencias y configurar el entorno de trabajo

In [1]:
#!pip install tensorflow-gpu==1.13.1

## Paso 2: Importar las dependencias

In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import fashion_mnist

tf.__version__

'2.3.0'

## Paso 3: Pre procesado de datos

### Cargar el dataset de FashionMNIST

In [3]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


### Normalizar las imágenes

In [4]:
X_train = X_train / 255.
X_test = X_test / 255.

### Redimensionar las imágenes

In [5]:
X_train.shape

(60000, 28, 28)

In [6]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [7]:
X_train.shape

(60000, 784)

## Paso 4: Construir el modelo

### Defining the model

In [8]:
model = tf.keras.models.Sequential()

### Definir el modelo

In [9]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

### Compilar el modelo

In [10]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Entrenar el modelo

In [11]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5372 - sparse_categorical_accuracy: 0.8104
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3999 - sparse_categorical_accuracy: 0.8548
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3671 - sparse_categorical_accuracy: 0.8660
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3456 - sparse_categorical_accuracy: 0.8727
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3303 - sparse_categorical_accuracy: 0.8796


### Evaluar el modelo

In [12]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Precisión del conjunto de test: {}".format(test_accuracy))

313/313 [==============================] - 0s 2ms/step - loss: 0.3670 - sparse_categorical_accuracy: 0.8648
Precisión del conjunto de test: 0.864799976348877


## Paso 5: Convertir el modelo a la versión de TensorFlow Lite

### Guardar el modelo

In [13]:
model_name = 'fashion_mobile_model.h5'
tf.keras.models.save_model(model, model_name)

### Crear el TFLite Converter

In [18]:
keras_model = tf.keras.models.load_model("/content/fashion_mobile_model.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)

### Convertir el modelo

In [19]:
tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmpcyj0yf8c/assets


### Guardar la versión de TFLite version del modelo

In [20]:
with open("tf_model.tflite", "wb") as f:
  f.write(tflite_model)